In [22]:
import pycalibration
pycalibration.install()

In [23]:
import numpy as np
import random
from sklearn.metrics.pairwise import pairwise_kernels, linear_kernel, polynomial_kernel,rbf_kernel,sigmoid_kernel,laplacian_kernel
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from pycalibration import ca
import time

In [473]:
def SKCE(kernel, predictions, outcomes):

    predictions = np.array(predictions)
    outcomes = np.array(outcomes)
    n = len(outcomes)

    # Initialize SKCE
    skce_matrix = np.zeros((n,n))

    # For each pair of data points
    for i in range(n):
        for j in range(i+1,n):
            # Compute the difference between unit vectors outcomes and predictions
            diff_Yi_g_Xi = np.array([predictions[i] - outcomes[i]])
            diff_Yj_g_Xj = np.array([predictions[j] - outcomes[j]])

            print(f"Shape of diff_Yi_g_Xi: {diff_Yi_g_Xi.shape}")
            print(f"Shape of diff_Yj_g_Xj: {diff_Yj_g_Xj.shape}")

            # Compute the kernel matrix for the pair
            k_g_Xi_g_Xj = kernel(predictions[i].reshape(1, -1), predictions[j].reshape(1, -1))

            matrix_kernel = k_g_Xi_g_Xj * np.eye(diff_Yi_g_Xi.shape[1])

            print(f"Shape of kernel: {matrix_kernel.shape}")


            #print(diff_Yi_g_Xi.T, matrix_kernel, diff_Yj_g_Xj)
            print("\n")
            # Compute the hi,j term
            skce_matrix[i,j] = (diff_Yi_g_Xi @ matrix_kernel) @ diff_Yj_g_Xj.T

    skce = np.sum(skce_matrix)

    
    # Divide the SKCE by n over 2
    prefactor = (n* (n-1)) // 2
    skce /= prefactor

    return skce


def transform_outcomes(predictions, outcomes):
    #creates vector for outcomes to match predictions shape
    outcomes_minus_one = [x - 1 for x in outcomes]
    transformed_outcomes = np.zeros_like(predictions)
    transformed_outcomes[np.arange(len(predictions)), outcomes_minus_one] = 1
    return transformed_outcomes

In [474]:
from sklearn.gaussian_process.kernels import RBF as RBF_GP
from sklearn.metrics.pairwise import rbf_kernel as RBF_PW

# Instantiate the RBF kernel from sklearn.gaussian_process.kernels
rbf_gp = RBF_GP(length_scale=1)

# Use the rbf_kernel function from sklearn.metrics.pairwise
def rbf_pw(X, Y=None, length_scale=1.0):
    gamma = 1.0 / (2 * length_scale ** 2)
    return RBF_PW(X, Y, gamma=gamma)

In [475]:
rng = np.random.default_rng(1234)
import random
rng = np.random.default_rng(1234)
predictions = [rng.dirichlet((3, 2, 5)) for _ in range(40)]
outcomes = rng.integers(low=1, high=4, size=40)
#outcomes = predictions.copy()
#kommt discrepanz von Kernel vs Impelentierung

#ECE bei Ensemble heterogene / homogene
#KCE vs. ECE ??


#print(transform_outcomes(predictions, outcomes))

# Now you can use these in your calculate_SKCE function
SKCE(rbf_pw, predictions, transform_outcomes(predictions, outcomes))
#print(SKCE(rbf_pw, predictions, outcomes))


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (1, 3)
Shape of kernel: (3, 3)


Shape of diff_Yi_g_Xi: (1, 3)
Shape of diff_Yj_g_Xj: (

0.1177590101389847

In [476]:
#skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
skce_julia = ca.SKCE(ca.tensor(ca.RBFKernel(), ca.ConstantKernel()), unbiased = True)
skce_julia(predictions, outcomes)

-0.0549087183263404

In [457]:
predictions = np.array([ 0.9, 0.1])
outcomes = np.array([True,False])

In [477]:
skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
skce_julia(predictions, outcomes)

RuntimeError: <PyCall.jlwrap (in a Julia function called from Python)
JULIA: MethodError: no method matching oneunit(::Type{Any})

Closest candidates are:
  oneunit(::Type{Union{Missing, T}}) where T
   @ Base missing.jl:105
  oneunit(::Type{T}) where T
   @ Base number.jl:370
  oneunit(::T) where T
   @ Base number.jl:369
  ...

Stacktrace:
  [1] oneunit(#unused#::Type{Any})
    @ Base .\missing.jl:106
  [2] _eval_start(d::Distances.SqEuclidean, #unused#::Type{Any}, #unused#::Type{Any}, #unused#::Nothing)
    @ Distances C:\Users\morit\.julia\packages\Distances\O1qLZ\src\metrics.jl:369
  [3] _eval_start(d::Distances.SqEuclidean, #unused#::Type{Any}, #unused#::Type{Any})
    @ Distances C:\Users\morit\.julia\packages\Distances\O1qLZ\src\metrics.jl:367
  [4] eval_start(d::Distances.SqEuclidean, a::Tuple{Vector{Float64}, Int64}, b::Tuple{Vector{Float64}, Int64})
    @ Distances C:\Users\morit\.julia\packages\Distances\O1qLZ\src\metrics.jl:366
  [5] _evaluate
    @ C:\Users\morit\.julia\p

In [ ]:
# Measure the time taken by function1
start_time = time.time()
SKCE(rbf_gp, predictions, outcomes)
end_time = time.time()
print(f"SKCE Python took {end_time - start_time} seconds to execute.")

# Measure the time taken by function2
start_time = time.time()
skce_julia(predictions, outcomes)
end_time = time.time()
print(f"SKCE Julia took {end_time - start_time} seconds to execute.")

In [117]:
import numpy as np

# Original 1D array
arr = np.array([0.05272924, -0.05272924])

# Reshape to a 2D array with a single row
arr_row = arr.reshape(1, -1)
print("Row array:")
print(arr_row)

# Reshape to a 2D array with a single column
arr_col = arr.reshape(-1, 1)
print("Column array:")
print(arr_col)


Row array:
[[ 0.05272924 -0.05272924]]
Column array:
[[ 0.05272924]
 [-0.05272924]]
